In [19]:
import random
import hashlib
from Crypto.Cipher import AES
import hmac

In [2]:
# y^2 = x^3 + a*x + b

In [3]:
def add_points(P, Q, p):
    x1, y1 = P
    x2, y2 = Q
    
    if x1 == x2 and y1 == y2:
        beta = (3*x1*x2 + a) * pow(2*y1, -1, p)
    else:
        beta = (y2 - y1) * pow(x2 - x1, -1, p)
    
    x3 = (beta*beta - x1 - x2) % p
    y3 = (beta * (x1 - x3) - y1) % p
    
    is_on_curve((x3, y3), p)
        
    return x3, y3

def is_on_curve(P, p):
    x, y = P
    assert (y*y) % p == ( pow(x, 3, p) + a*x + b ) % p
    
def apply_double_and_add_method(G, k, p):
    target_point = G
    
    k_binary = bin(k)[2:] #0b1111111001
    
    for i in range(1, len(k_binary)):
        current_bit = k_binary[i: i+1]
        
        # doubling - always
        target_point = add_points(target_point, target_point, p)
        
        if current_bit == "1":
            target_point = add_points(target_point, G, p)
    
    is_on_curve(target_point, p)
    
    return target_point

In [4]:
# Secp256k1
a = 0; b = 7
G = (55066263022277343669578718895168534326250603453777594175500187360389116729240, 
     32670510020758816978083085130507043184471273380659243275938904335757337482424)

p = pow(2, 256) - pow(2, 32) - pow(2, 9) - pow(2, 8) - pow(2, 7) - pow(2, 6) - pow(2, 4) - pow(2, 0)
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337

In [5]:
is_on_curve(G, p)

# Alice generates her private and public key

In [6]:
# private key
ka = random.getrandbits(256)

# public key
Q = apply_double_and_add_method(G = G, k = ka, p = p)

# Bob generates a random key

In [7]:
rb = random.getrandbits(256)

# public - send this point to Alice
U = apply_double_and_add_method(G = G, k = rb, p = p)

# private - keeps secret
T = apply_double_and_add_method(G = Q, k = rb, p = p)

# Key Derivation Function (Public)

In [20]:
def derive_keys(T):
    tx, ty = T
    
    tx_binary = bin(tx)[2:]
    
    #192-bits
    tx_binary_cropped = tx_binary[0:192]
    
    tx_restored = int(tx_binary_cropped, 2)
    
    #sha-256
    hash_hex = hashlib.sha256(str.encode(str(tx_restored))).hexdigest()
    hash_binary = bin(int(hash_hex, 16))[2:]
    
    k1 = int(hash_binary[0:128], 2).to_bytes(16, byteorder="big")
    k2 = int(hash_binary[128:], 2).to_bytes(16, byteorder="big")
    
    return k1, k2

def find_mac(message, key):
    return hmac.new(key, message, hashlib.sha256).hexdigest()

In [12]:
k1, k2 = derive_keys(T)

# Encryption

In [15]:
msg = "attack tomorrow!"

In [16]:
obj_bob = AES.new(k1)

In [17]:
c = obj_bob.encrypt(msg)

In [18]:
c

b',bgt\xd26\x84\xd5\xa3\xad"d\x11\x95\x98\x1e'

In [21]:
r = find_mac(message = c, key=k2)

In [22]:
(U, c, r)

((74167907263546634922179378596423937735743665773803862171882405761903892922303,
  52034086156418309202011483943380820869379888355606836194274686146628789809265),
 b',bgt\xd26\x84\xd5\xa3\xad"d\x11\x95\x98\x1e',
 '2e0456b115686af2e5d791a2529d19411f816b77bb2b12ea8dab9eef4bc204ef')

# Decryption

In [ ]:
# (U, c, r)

In [23]:
T_prime = apply_double_and_add_method(G = U, k = ka, p = p)

In [24]:
assert T_prime == T

In [25]:
k1_prime, k2_prime = derive_keys(T_prime)

In [26]:
r_prime = find_mac(message = c, key = k2_prime)

In [27]:
assert r_prime == r

In [29]:
obj_alice = AES.new(k1_prime)
plaintext = obj_alice.decrypt(c)

In [30]:
plaintext

b'attack tomorrow!'